In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

MIN_TRANSFORMERS_VERSION = '4.25.1'

# check transformers version
assert transformers.__version__ >= MIN_TRANSFORMERS_VERSION, f'Please upgrade transformers to version {MIN_TRANSFORMERS_VERSION} or higher.'


In [ ]:
# init
# tokenizer = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Instruct-3B-v1")
# model = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Instruct-3B-v1", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Instruct-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Instruct-7B-v0.1", torch_dtype=torch.float16)

In [ ]:
# torch.cuda.is_available()

In [ ]:
# import os
# 
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [ ]:
device = torch.device("cuda:1")
# device = torch.device("cpu")

In [ ]:
import torch.nn as nn


# model= nn.parallel.DistributedDataParallel(model)
model = model.to(device)


In [ ]:
# infer
# prompt = "Q: The capital of France is?\nA:"
# prompt = "Q: The capital of Japan is?\nA:"
prompt = "Q: 日本の首都は？\nA:"
inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
input_length = inputs.input_ids.shape[1]
outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    min_length=1,
    do_sample=True,
    temperature=0.01,
    top_p=0.7,
    top_k=50,
    return_dict_in_generate=True,
    repetition_penalty=1.11,
    length_penalty=0.5,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)
token = outputs.sequences[0, input_length:]
output_str = tokenizer.decode(token)

In [ ]:
print(output_str.split("\n\n")[0])

# """
# Paris
# """


In [ ]:
tokenizer.eos_token_id

In [ ]:
prompt = "a=1, b='d', c='x' をJSON形式にしてください。"
inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
input_length = inputs.input_ids.shape[1]
outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    min_length=1,
    do_sample=True,
    temperature=0.01,
    top_p=0.7,
    top_k=50,
    return_dict_in_generate=True,
    repetition_penalty=1.2,
    length_penalty=0.5,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)
token = outputs.sequences[0, input_length:]
output_str = tokenizer.decode(token)

# print(output_str.split("\n\n")[0])
print(len(output_str))
print("-" * 100)
print(output_str)

In [ ]:
prompt = "カニを食べたいのですよー"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model(**inputs)


In [ ]:
o = model.gpt_neox(**inputs)
o.last_hidden_state.shape       # (B, S, D)